In [2]:
import pandas as pd
import seaborn as sns
import os

# Время и сводные таблицы: Дополнительный проект

# В папке находятся файлы с продажами продуктов через телемаркетинг. Каждый файл содержит, как минимум, 4 колонки (поля): FILIAL_ID, SUBS_ID, PROD_ID, ACT_DTTM.

## Суть задачи в том, чтобы проверить подключения продуктов определенным пользователям, соединив файлы о продажах с логами по подключениям в системе.

### Особенности данных:

### 1. сотрудники телемаркетинга не всегда указывают полный id, если 'id' нет в начале SUBS_ID, то нужно его добавить
### 2. поля в файлах могут быть расположены абсолютно случайным образом, но названия полей статичны
### 3. продажа не засчитывается, если отключение (END_DTTM) произошло меньше чем через 5 минут после подключения (START_DTTM)
### 4. если в файле с продажами встречается строка без указанного SUBS_ID, она пропускается
#### Сохраните результат в csv файл с разделителем ;, содержащий корректные подключения.

In [44]:
path_to_files = ('D:/курс_архив/my_files/shared/homeworks/python_ds_miniprojects/5_subsid/subsid/')

In [45]:
path_to_files

'D:/курс_архив/my_files/shared/homeworks/python_ds_miniprojects/5_subsid/subsid/'

In [46]:
# Чтение файла с логами подключений продуктов
prod_activations_logs = pd.read_csv('D:курс_архив/my_files/shared/homeworks/python_ds_miniprojects/5_subsid/subsid/prod_activations_logs.csv', sep=';')

In [47]:
# Сбрасываем индекс, чтобы он был непрерывным
prod_activations_logs = prod_activations_logs.reset_index(drop=True)

In [48]:
prod_activations_logs

,SUBS_ID,PROD_ID,START_DTTM,END_DTTM
0,id4651830,1954,20-03-2020 14:59,01-12-2020 00:00
1,id7646509,6431,19-03-2020 13:00,19-03-2020 13:03
2,id7461794,3310,20-03-2020 17:25,01-12-2020 00:00
3,id5416547,1743,17-03-2020 10:17,25-03-2020 11:00
4,id8238421,1859,01-03-2020 11:42,01-03-2020 11:43
5,id8641743,2752,20-03-2020 15:44,21-04-2020 15:44
6,id2185490,3210,16-03-2020 16:28,01-12-2020 00:00
7,id7642700,3020,15-03-2020 14:21,15-03-2020 23:42
8,id8741631,5677,19-03-2020 12:28,01-12-2020 00:00


In [49]:
# Функция для добавления 'id' в начало SUBS_ID, если его там нет
def add_id(x):
    if x.startswith('id'):
        x = x
    else:
        x = 'id' + x
    return x 

In [50]:
# Создаём пустой DataFrame для объединения всех файлов с продажами
# Цикл по всем файлам в папке
# Проверяем, что файл начинается с "tm" (файлы продаж)
# Чтение каждого файла продаж, удаляем строки с пустым SUBS_ID
# Объединяем текущий файл с общим DataFrame
# Сбрасываем индексы после каждого объединения
# Применяем функцию add_id ко всем значениям в колонке SUBS_ID
full_df = pd.DataFrame()
for i in os.listdir(path_to_files):
    if i.startswith("tm"):
        df_tm = pd.read_csv(path_to_files + i,sep=';').dropna(subset=["SUBS_ID"])
        full_df = pd.concat([full_df, df_tm])
        full_df = full_df.reset_index(drop=True)
        full_df['SUBS_ID'] = full_df.SUBS_ID.apply(add_id)    

In [51]:
# Проверяем итоговый DataFrame
full_df

,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM
0,id4651830,1,1954,20-03-2020 14:59
1,id7646509,5,6431,19-03-2020 13:00
2,id7412683,4,3313,22-03-2020 17:25
3,id5416547,3,1743,17-03-2020 10:17
4,id8362218,7,9879,05-03-2020 11:42
5,id2185490,2,3210,16-03-2020 16:28
6,id5764122,3,1499,18-03-2020 15:44
7,id7642700,6,3020,15-03-2020 14:21
8,id1374509,2,5677,17-03-2020 11:48


In [52]:
# Проверяем типы данных в таблице
full_df.dtypes

SUBS_ID      object
FILIAL_ID     int64
PROD_ID       int64
ACT_DTTM     object
dtype: object

In [53]:
# Объединяем DataFrame с продажами и логами подключений по ключам SUBS_ID и PROD_ID
full_df = full_df.merge(prod_activations_logs, how='inner', on=['SUBS_ID', 'PROD_ID'])

In [54]:
# Ещё раз проверяем типы данных
full_df.dtypes

SUBS_ID       object
FILIAL_ID      int64
PROD_ID        int64
ACT_DTTM      object
START_DTTM    object
END_DTTM      object
dtype: object

In [55]:
# Преобразуем столбцы START_DTTM и END_DTTM в формат datetime
full_df['START_DTTM'] = pd.to_datetime(full_df.START_DTTM, format='%d-%m-%Y %H:%M')

In [56]:
# Преобразуем столбцы START_DTTM и END_DTTM в формат datetime
full_df['END_DTTM'] = pd.to_datetime(full_df.END_DTTM, format='%d-%m-%Y %H:%M')

In [57]:
# Проверка типов данных после преобразования
full_df.dtypes

SUBS_ID               object
FILIAL_ID              int64
PROD_ID                int64
ACT_DTTM              object
START_DTTM    datetime64[ns]
END_DTTM      datetime64[ns]
dtype: object

In [58]:
# Выводим DataFrame для просмотра
full_df

,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM,START_DTTM,END_DTTM
0,id4651830,1,1954,20-03-2020 14:59,2020-03-20 14:59:00,2020-12-01 00:00:00
1,id7646509,5,6431,19-03-2020 13:00,2020-03-19 13:00:00,2020-03-19 13:03:00
2,id5416547,3,1743,17-03-2020 10:17,2020-03-17 10:17:00,2020-03-25 11:00:00
3,id2185490,2,3210,16-03-2020 16:28,2020-03-16 16:28:00,2020-12-01 00:00:00
4,id7642700,6,3020,15-03-2020 14:21,2020-03-15 14:21:00,2020-03-15 23:42:00


In [17]:
# Создаём новую колонку difference, которая содержит разницу между END_DTTM и START_DTTM
full_df['difference'] = full_df.END_DTTM - full_df.START_DTTM

In [18]:
full_df

,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM,START_DTTM,END_DTTM,difference
0,id4651830,1,1954,20-03-2020 14:59,2020-03-20 14:59:00,2020-12-01 00:00:00,255 days 09:01:00
1,id7646509,5,6431,19-03-2020 13:00,2020-03-19 13:00:00,2020-03-19 13:03:00,0 days 00:03:00
2,id5416547,3,1743,17-03-2020 10:17,2020-03-17 10:17:00,2020-03-25 11:00:00,8 days 00:43:00
3,id2185490,2,3210,16-03-2020 16:28,2020-03-16 16:28:00,2020-12-01 00:00:00,259 days 07:32:00
4,id7642700,6,3020,15-03-2020 14:21,2020-03-15 14:21:00,2020-03-15 23:42:00,0 days 09:21:00


In [19]:
# Сортируем DataFrame по колонке SUBS_ID
full_df.sort_values('SUBS_ID')

,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM,START_DTTM,END_DTTM,difference
3,id2185490,2,3210,16-03-2020 16:28,2020-03-16 16:28:00,2020-12-01 00:00:00,259 days 07:32:00
0,id4651830,1,1954,20-03-2020 14:59,2020-03-20 14:59:00,2020-12-01 00:00:00,255 days 09:01:00
2,id5416547,3,1743,17-03-2020 10:17,2020-03-17 10:17:00,2020-03-25 11:00:00,8 days 00:43:00
4,id7642700,6,3020,15-03-2020 14:21,2020-03-15 14:21:00,2020-03-15 23:42:00,0 days 09:21:00
1,id7646509,5,6431,19-03-2020 13:00,2020-03-19 13:00:00,2020-03-19 13:03:00,0 days 00:03:00


In [34]:
# Применяем фильтр: оставляем только строки, где разница больше 5 минут
filtered_df = full_df[full_df['difference'] > pd.Timedelta('5 minutes')]

In [39]:
filtered_df.sort_values('difference', ascending=False)

,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM,START_DTTM,END_DTTM,difference
3,id2185490,2,3210,16-03-2020 16:28,2020-03-16 16:28:00,2020-12-01 00:00:00,259 days 07:32:00
0,id4651830,1,1954,20-03-2020 14:59,2020-03-20 14:59:00,2020-12-01 00:00:00,255 days 09:01:00
2,id5416547,3,1743,17-03-2020 10:17,2020-03-17 10:17:00,2020-03-25 11:00:00,8 days 00:43:00
4,id7642700,6,3020,15-03-2020 14:21,2020-03-15 14:21:00,2020-03-15 23:42:00,0 days 09:21:00
